In [1]:
import numpy as np
from numpy import loadtxt
import fitsio
import healpy as hp
import healsparse as hsp
import matplotlib.pyplot as plt
import Config

In [13]:
dirTrain = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/'
numBins = 1000
conditions = Config.conditions
res = 4096

In [3]:
validPix = fitsio.read(dirTrain + 'Valid_4096_Pixels.fits')['PIXEL']
pixCheck = np.full(12*(4096**2), False, dtype = bool)
pixCheck[validPix] = True

In [4]:
stds = np.loadtxt(dirTrain + 'Standard_Deviations.csv', delimiter = ',')
mean = np.loadtxt(dirTrain + 'Means.csv', delimiter = ',')
inds = np.loadtxt(dirTrain + 'Indices.csv', delimiter = ',').astype(int)

In [5]:
strFeatVec = loadtxt(dirTrain + 'Feature_Vectors.csv', delimiter = '\t', dtype = object)
featVec = []
for i in range(len(strFeatVec)):
    
    indVec = []
    
    for j in range(len(strFeatVec[i])):
        
        indVec.append(float(strFeatVec[i][j]))
        
    indVec = np.array(indVec)
    featVec.append(indVec)
    
featVec = np.array(featVec, dtype = object)

In [6]:
# This is the actual file containing all of the data
balrFile = '/afs/hep.wisc.edu/bechtol-group/MegansThings/balrog_detection_catalog_sof_run2_stars_v1.4_avg_added_match_flags.fits'
# This reads in all of the data. Most of these are just flags, the only pieces that get used much outside
# of filtering are detected, true_ra and true_dec which get used to convert into healPixels.
balrData = fitsio.read(balrFile, columns = ['detected', 'true_ra', 'true_dec',
                                            'flags_foreground', 'flags_badregions', 'flags_footprint',
                                            'match_flag_1.5_asec'])

In [7]:
# These are in degrees which is why lonlat is set to True in the next cell.
balrRA = balrData['true_ra']
balrDEC = balrData['true_dec']
# This is used for detection rates, each point is either a 0 (no detection) or a 1 (detection)
balrDETRepeats = balrData['detected']
# Everything from here on out is simply used in order to filter the data
FOREGROUND = balrData['flags_foreground']
BADREGIONS = balrData['flags_badregions']
FOOTPRINT = balrData['flags_footprint']
ARCSECONDS = balrData['match_flag_1.5_asec']

# This is used to filter out any injections that either weren't detected or had flags raised.
cutIndices = np.where((FOREGROUND == 0) & 
                      (BADREGIONS < 2) & 
                      (FOOTPRINT == 1) & 
                      (ARCSECONDS < 2))[0]

# This reduced the data down to the actually valid pixels.
balrDETRepeats = balrDETRepeats[cutIndices]
balrRA = balrRA[cutIndices]
balrDEC = balrDEC[cutIndices]

# This converts the RA and DEC values from above to healpixels so we can compare to the sky condition.
balrPIXRepeats = hp.ang2pix(res, balrRA, balrDEC, lonlat = True, nest = True)

# This sorts by the pixel in order to make following methods more efficient.
sortInds = balrPIXRepeats.argsort()
balrPIXRepeats = balrPIXRepeats[sortInds[::1]]
balrDETRepeats = balrDETRepeats[sortInds[::1]]

# These are indices that will be looping through the pixStar and starPix arrays in parallel.
uniqInd = 0
balrInd = 0

# This will be used to store the number of stars at each pixel.
balrPIX = np.unique(balrPIXRepeats) # The unique pixels, with no repeats.
balrDET = np.zeros_like(balrPIX)
balrINJ = np.zeros_like(balrPIX)

while balrInd < len(balrPIXRepeats):
    if balrPIX[uniqInd] == balrPIXRepeats[balrInd]: # If the pixels match up in the arrays.
        balrDET[uniqInd] += balrDETRepeats[balrInd]     # Add one if there was a detection at this location.
        balrINJ[uniqInd] += 1                # Add one to the corresponding spot in the balStar array.
        balrInd += 1                         # Add one to the starInd to see if the next index in starPix is also the same.
        # Since the last index of pixStar and starPix are the same, starInd will increase the last time through the loop,
        # making this the index that we must restrict in the while loop.
    else:
        uniqInd += 1 # If the pixels are no longer the same, increase the index you check in the pixStar array.

validIndices = np.full(len(balrPIX), False, dtype = bool)
validIndices[np.where(pixCheck[balrPIX])[0]] = True
    
# Degrades all of the values to a common set of pixels
balrPIX = balrPIX[validIndices]
balrDET = balrDET[validIndices]
balrINJ = balrINJ[validIndices]

# Stores the original data for later comparisons
originalBalrDET = balrDET
originalBalrINJ = balrINJ
aveEff = np.sum(originalBalrDET) / np.sum(originalBalrINJ)

In [8]:
condMaps = []

# This loops over every condition file except for stellar density which has a different format
for i in range(len(conditions)):
    condData = fitsio.read(dirTrain + conditions[i] + '.fits') # This reads in the data
    condSigExt = np.full(12*(4096**2), 0.0) # Gives a default value
    condSigExt[validPix] = condData['SIGNAL'] # Changes all valid pixels to their corresponding signals
    condMaps.append(condSigExt[balrPIX]) # Only stores the values that are in pixels with injections

condMaps = np.array(condMaps, dtype = object) # Converts to an array

In [9]:
stanMaps = []

for i in range(len(condMaps)):
    stanMaps.append((condMaps[i] - mean[i]) / stds[i])

stanMaps = np.array(stanMaps, dtype = object) # Converts to an array

In [10]:
redMaps = np.matmul(featVec, stanMaps)

In [14]:
incPix = np.full(len(balrPIX), True, dtype = bool)

for ind in inds:
    
    oneMap = redMaps[ind]

    binIndLims = [0]

    for i in range(numBins):
        binIndLims.append(int((len(oneMap) - binIndLims[-1]) / (numBins - i)) + (binIndLims[-1]))

    sortInds = oneMap.argsort()
    oneMapTemp = oneMap[sortInds[::1]]
    injStarTemp = balrINJ[sortInds[::1]]

    injBins = []
    mapBins = []
    
    for i in range(numBins):
        injBins.append(injStarTemp[binIndLims[i]:binIndLims[i+1]])
        mapBins.append(oneMapTemp[binIndLims[i]:binIndLims[i+1]])

    xValues = []

    for i in range(numBins):
        xValues.append(np.sum(mapBins[i] * injBins[i]) / np.sum(injBins[i]))

    xValues = np.array(xValues)
    
    incPix[np.where((oneMap < xValues[0]) | (oneMap > xValues[-1]))[0]] = False

In [15]:
print(np.sum(incPix)/len(incPix))

0.9833262918710901


In [16]:
len(inds)

30